In [1]:
import glob
import os

image_dir = r"C:\Users\molly\Downloads\Dec 3 Images"
image_paths = glob.glob(os.path.join(image_dir, "*.*"))

print("Number of images found:", len(image_paths))
print(image_paths[:3])  # Show first few


Number of images found: 12
['C:\\Users\\molly\\Downloads\\Dec 3 Images\\IMG_1224.HEIC', 'C:\\Users\\molly\\Downloads\\Dec 3 Images\\IMG_1234.HEIC', 'C:\\Users\\molly\\Downloads\\Dec 3 Images\\IMG_1238.HEIC']


In [2]:
import sys

# Install all the packages needed for this assignment
!{sys.executable} -m pip install ultralytics opencv-python pillow pandas scikit-learn


In [3]:
import sys
print(sys.executable)

!{sys.executable} -m pip install ultralytics


C:\Users\molly\anaconda3\python.exe


In [5]:
import os
import glob
import time

import numpy as np
import pandas as pd

from ultralytics import YOLO
import cv2
from sklearn.cluster import KMeans


# 1. SET IMAGE FOLDER

# Change this to your folder path if needed
image_dir = r"C:\Users\molly\Downloads\Dec 3 Images"

if not os.path.exists(image_dir):
    raise FileNotFoundError(f"Image folder not found: {image_dir}")

# Get all image files (jpg, jpeg, png)
image_paths = sorted(
    glob.glob(os.path.join(image_dir, "*.jpg"))
    + glob.glob(os.path.join(image_dir, "*.jpeg"))
    + glob.glob(os.path.join(image_dir, "*.png"))
)

print(f"Found {len(image_paths)} images.")
if len(image_paths) < 1:
    raise ValueError("No images found. Add at least one .jpg/.png image to the folder and rerun.")

# ==============================
# 2. PART A (i): OBJECT DETECTION WITH TWO MODELS
# ==============================

# Load two different deep learning object detection models
model_a = YOLO("yolov8n.pt")  # Model 1: smaller, faster
model_b = YOLO("yolov8s.pt")  # Model 2: larger, usually more accurate

def run_model_on_image(model, model_name, image_path):
    """
    Run an object detection model on a single image and return:
    - image_name
    - model name
    - time taken (seconds)
    - number of objects detected
    - average confidence
    - summary of class counts (e.g., 'person:2, car:1')
    """
    img_name = os.path.basename(image_path)

    # Measure time
    start_time = time.time()
    results = model(image_path)  # YOLO prediction
    elapsed = time.time() - start_time

    # YOLOv8 detections
    boxes = results[0].boxes
    n_objects = len(boxes)

    # Probabilities (confidences)
    if n_objects > 0:
        confidences = boxes.conf.cpu().numpy().tolist()
        avg_conf = float(np.mean(confidences))
    else:
        confidences = []
        avg_conf = 0.0

    # Class counts per type (person, car, etc.)
    class_counts = {}
    if n_objects > 0:
        cls_ids = boxes.cls.cpu().numpy().tolist()
        for cid in cls_ids:
            cid = int(cid)
            class_name = results[0].names[cid]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

    # Turn class_counts dict into a short text string
    if class_counts:
        class_summary = ", ".join(f"{k}:{v}" for k, v in class_counts.items())
    else:
        class_summary = ""

    return {
        "image_name": img_name,
        "model": model_name,
        "time_sec": round(elapsed, 4),
        "num_objects": n_objects,
        "avg_confidence": round(avg_conf, 4),
        "class_counts": class_summary
    }

# Run both models on each image and collect results
detection_results = []

for img_path in image_paths:
    # Model A
    res_a = run_model_on_image(model_a, "yolov8n", img_path)
    detection_results.append(res_a)

    # Model B
    res_b = run_model_on_image(model_b, "yolov8s", img_path)
    detection_results.append(res_b)

# Create table for Part A (i)
df_detection = pd.DataFrame(detection_results)

print("Part A (i) – Object Detection Comparison Table")
print(df_detection)

# Save for use in your Word document
df_detection.to_csv("object_detection_comparison.csv", index=False)
print("\nSaved object_detection_comparison.csv in the current folder.")

# ==============================
# 3. PART A (ii): EXTRA INFORMATION FROM IMAGES
#    Example: DOMINANT COLOR (R,G,B) PER IMAGE
# ==============================

def get_dominant_color(image_path, k=3):
    """
    Find the dominant RGB color in an image using K-Means clustering.
    Returns (R, G, B) as integers.
    This does NOT use deep learning, just basic image processing.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")

    # Convert from BGR (OpenCV default) to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resize to 100x100 to speed up computation
    img_small = cv2.resize(img, (100, 100), interpolation=cv2.INTER_AREA)

    # Flatten to list of pixels
    pixels = img_small.reshape(-1, 3)

    # K-Means clustering on colors
    kmeans = KMeans(n_clusters=k, n_init=5, random_state=42)
    kmeans.fit(pixels)

    centers = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_

    # Find the largest color cluster
    unique_labels, counts = np.unique(labels, return_counts=True)
    dominant_idx = unique_labels[np.argmax(counts)]

    dominant_color = tuple(centers[dominant_idx])  # (R, G, B)
    return dominant_color

extra_results = []

for img_path in image_paths:
    img_name = os.path.basename(img_path)
    dom_color = get_dominant_color(img_path, k=3)
    extra_results.append({
        "image_name": img_name,
        "dominant_color_rgb": dom_color
    })

df_extra = pd.DataFrame(extra_results)

print("\nPart A (ii) – Extra Info (Dominant Color)")
print(df_extra)

# Save for your Word document
df_extra.to_csv("extra_image_features.csv", index=False)
print("\nSaved extra_image_features.csv in the current folder.")


Found 8 images.

image 1/1 C:\Users\molly\Downloads\Dec 3 Images\IMG_6788.jpg: 640x480 2 carrots, 1 dining table, 70.4ms
Speed: 4.2ms preprocess, 70.4ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\molly\Downloads\Dec 3 Images\IMG_6788.jpg: 640x480 4 carrots, 1 dining table, 156.4ms
Speed: 3.0ms preprocess, 156.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\molly\Downloads\Dec 3 Images\IMG_6791.jpg: 480x640 6 persons, 1 cup, 1 dining table, 64.5ms
Speed: 2.6ms preprocess, 64.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\molly\Downloads\Dec 3 Images\IMG_6791.jpg: 480x640 5 persons, 1 cup, 1 dining table, 138.7ms
Speed: 3.6ms preprocess, 138.7ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\molly\Downloads\Dec 3 Images\IMG_6794.jpg: 640x480 3 persons, 63.8ms
Speed: 2.6ms preprocess, 63.8ms inference, 3.0ms postprocess per imag